In [52]:
from Bio import Entrez
import pandas as pd
from tqdm.notebook import tqdm

Entrez.email = "singhmohit.1889@gmail.com"
Entrez.api_key = "1c22910bfa9f49693b451d3dfec15c1dbb09"

def extract_pubmed_id(reference):
    try:
        for article_id in reference['ArticleIdList']:
            if article_id.attributes['IdType'] == 'pubmed':
                return str(article_id)
    except:
        print(reference)
    return float('NaN')

def fetch_paper_details(pmid):
    handle = Entrez.efetch(db="pubmed", id=pmid, rettype="xml", retmode="text")
    records = Entrez.read(handle)
    handle.close()
    
    paper_title = pmid
#     paper_title = records['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
    authors = records['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList']
    paper_authors = [f"{author['ForeName']} {author['LastName']}" for author in authors]
    journal = records['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['Title']
    year = records['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year']   
    return paper_title, paper_authors, journal, year

def fetch_citations(pmid):
    handle = Entrez.elink(dbfrom="pubmed", id=pmid, linkname="pubmed_pubmed_citedin")
    records = Entrez.read(handle)
    handle.close()
    
    citing_paper_titles = []
    citing_paper_references = []
    details = []
    if records[0]["LinkSetDb"]:
        for link in records[0]["LinkSetDb"][0]["Link"]:
            citing_pmid = link["Id"]
            handle = Entrez.efetch(db="pubmed", id=citing_pmid, rettype="xml", retmode="text")
            records = Entrez.read(handle)
            handle.close()

#             paper_title = records['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
            paper_title = citing_pmid
            references = records['PubmedArticle'][0]['PubmedData']['ReferenceList'][0]['Reference']
            reference_list = [extract_pubmed_id(reference) for reference in references]
            details.append((paper_title,reference_list))
    
    return details

In [6]:
df = pd.read_csv("savedrecs imp csv.csv",encoding='latin-1')
PubmedId = df[~df['Pubmed Id'].isna()]['Pubmed Id'].values

In [3]:
# PubmedId = PubmedId.astype(int)
# PubmedId = PubmedId.astype(str)

In [3]:
PubmedId = ['34910933',
'7546706',
# '32998157',
# '26264835',
# '25233993',
# '18797453',
# '20457754',
# '22334017',
# '29739835',
# '33414215',
# '32783919'
           ]

In [53]:
PubmedId = ['37371829','37020354']

In [54]:
df = pd.DataFrame([],columns=['Paper_Title','Paper_Authors','Journal','Year','Citing_Paper_Title','Citing_Paper_References'])
for pmid in tqdm(PubmedId):
    
    paper_title, paper_authors, journal, year = fetch_paper_details(pmid)
    citing_papers = fetch_citations(pmid)
    for title, ref in citing_papers:
        df.loc[len(df)] = [paper_title, paper_authors, journal, year,title,ref]
#     except Exception as e:
#         print(pmid,e)

  0%|          | 0/2 [00:00<?, ?it/s]

{'Citation': 'Kozai T, Fernandez-Martinez J, van Eeuwen T, Gallardo P, Kapinos LE, Mazur A, Zhang W, Tempkin J, Panatala R, Delgado-Izquierdo M, et al: Dynamic molecular mechanism of the nuclear pore complex permeability barrier. bioRxiv 2023.'}
{'Citation': 'Pérez-Berlanga M, Wiersma VI, Zbinden A, De Vos L, Wagner U, Foglieni C, Mallona I, Betz KM, Cléry A, Weber J, et al: TDP-43 oligomerization and RNA binding are codependent but their loss elicits distinct pathologies. bioRxiv 2022:2022.2005.2023.493029.'}
{'Citation': 'Lee YJ, Rio DC: Analysis of altered pre-mRNA splicing patterns caused by a mutation in the RNA binding protein hnRNPA1 linked to amyotrophic lateral sclerosis. bioRxiv 2022:2022.2002.2003.479052.'}
{'Citation': 'Aikio M, Wobst HJ, Odeh HM, Lee B-L, Class B, Ollerhead TA, Mack KL, Ford AF, Barbieri EM, Cupo RR, et al: Opposing roles of p38α-mediated phosphorylation and arginine methylation in driving TDP-43 proteinopathy. bioRxiv 2021:2021.2008.2004.455154.'}
{'Citat

In [55]:
df

,Paper_Title,Paper_Authors,Journal,Year,Citing_Paper_Title,Citing_Paper_References
0,37371829,"[Owen Daly, Azita Josefine Mahiny, Sara Majesk...",Biomedicines,2023,38400169,"[35821637, 37705354, 36311701, 37595605, 37454..."
1,37371829,"[Owen Daly, Azita Josefine Mahiny, Sara Majesk...",Biomedicines,2023,38254150,"[33049211, 33942880, 27498188, 28762175, 34706..."
2,37020354,"[Jennifer Cable, Barney S Graham, Richard A Ko...",Annals of the New York Academy of Sciences,2023,38125026,"[31869338, 26229114, 27153120, 25121750, 21987..."
3,37020354,"[Jennifer Cable, Barney S Graham, Richard A Ko...",Annals of the New York Academy of Sciences,2023,37084166,"[nan, nan, 35454874, nan, 35089462, nan, 36675..."


In [26]:
handle = Entrez.elink(dbfrom="pubmed", id='37371829', linkname="pubmed_pubmed_citedin")
records = Entrez.read(handle)
handle.close()

citing_paper_titles = []
citing_paper_references = []
details = []
if records[0]["LinkSetDb"]:
    for link in records[0]["LinkSetDb"][0]["Link"]:
        citing_pmid = link["Id"]
        print(citing_pmid)
        handle = Entrez.efetch(db="pubmed", id=citing_pmid, rettype="xml", retmode="text")
        records = Entrez.read(handle)
        handle.close()

        paper_title = records['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
        references = records['PubmedArticle'][0]['PubmedData']['ReferenceList'][0]['Reference']
        reference_list = [reference['Citation'] for reference in references]
        details.append((paper_title,reference_list))
else:
    print('Hi')

38400169
38254150


In [27]:
handle = Entrez.efetch(db="pubmed", id='38400169', rettype="xml", retmode="text")
records = Entrez.read(handle)
handle.close()

In [30]:
records['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']

'Lipid Nanoparticle (LNP) Delivery Carrier-Assisted Targeted Controlled Release mRNA Vaccines in Tumor Immunity.'

In [39]:
a = records['PubmedArticle'][0]['PubmedData']['ReferenceList'][0]['Reference'][0]#['ArticleIdList']
a

{'Citation': 'Bevers S., Kooijmans S.A.A., Van de Velde E., Evers M.J.W., Seghers S., Gitz-Francois J.J.J.M., van Kronenburg N.C.H., Fens M.H.A.M., Mastrobattista E., Hassler L., et al. mRNA-LNP vaccines tuned for systemic immunization induce strong antitumor immunity by engaging splenic immune cells. Mol. Ther. 2022;30:3078–3094. doi: 10.1016/j.ymthe.2022.07.007.', 'ArticleIdList': [StringElement('10.1016/j.ymthe.2022.07.007', attributes={'IdType': 'doi'}), StringElement('PMC9273295', attributes={'IdType': 'pmc'}), StringElement('35821637', attributes={'IdType': 'pubmed'})]}

In [38]:
def extract_pubmed_id(reference):
    for article_id in reference['ArticleIdList']:
        if article_id.attributes['IdType'] == 'pubmed':
            return str(article_id)
    return None

In [40]:
extract_pubmed_id(a)

'35821637'

In [5]:
df.to_csv('output2.csv',index=False)

In [ ]:
34910933 'AuthorList'
32998157 list index out of range
25233993 list index out of range
20457754 list index out of range
32783919

In [9]:
df1 = pd.read_csv('output.csv')
df2 = pd.read_csv('output1.csv')

In [10]:
df = pd.concat([df1,df2])

In [11]:
df

,Paper_Title,Paper_Authors,Journal,Year,Citing_Paper_Title,Citing_Paper_References
0,Progress in vaccine development for infectious...,"['Jennifer Cable', 'Barney S Graham', 'Richard...",Annals of the New York Academy of Sciences,2023,Oral tolerance to systemic vaccination remains...,"['Cram J.A., Fiore-Gartland A.J., Srinivasan S..."
1,Progress in vaccine development for infectious...,"['Jennifer Cable', 'Barney S Graham', 'Richard...",Annals of the New York Academy of Sciences,2023,Extracellular microvesicles: biologic properti...,"['Adnani L, Spinelli C, Tawil N, Rak J (2022) ..."
2,Modified uridines are the key to a successful ...,['Katalin Karikó'],Nature reviews. Immunology,2021,Stabilized 5' Cap Analogue for Optochemical Ac...,['Curreri A.; Sankholkar D.; Mitragotri S.; Zh...
3,Modified uridines are the key to a successful ...,['Katalin Karikó'],Nature reviews. Immunology,2021,Transfection of hypoxia-inducible factor-1α mR...,"['Sen CK, Gordillo GM, Roy S, Kirsner R, Lambe..."
4,Modified uridines are the key to a successful ...,['Katalin Karikó'],Nature reviews. Immunology,2021,Race with virus evolution: The development and...,"['Wolff J.A., Malone R.W., Williams P., Chong ..."
...,...,...,...,...,...,...
1615,A noninflammatory mRNA vaccine for treatment o...,"['Christina Krienke', 'Laura Kolb', 'Elif Dike...","Science (New York, N.Y.)",2021,<i>In vitro</i> and <i>ex vivo</i> functional ...,"['Todd J.A., Acha-Orbea H., Bell J.I., Chao N...."
1616,A noninflammatory mRNA vaccine for treatment o...,"['Christina Krienke', 'Laura Kolb', 'Elif Dike...","Science (New York, N.Y.)",2021,Human Vaccines & Immunotherapeutics: news.,"['Rappazzo CG, Tse LV, Kaku CI, Wrapp D, Sakha..."
1617,A noninflammatory mRNA vaccine for treatment o...,"['Christina Krienke', 'Laura Kolb', 'Elif Dike...","Science (New York, N.Y.)",2021,SARS-CoV-2 vaccines in patients with SLE.,['Johns Hopkins University coronavirus resourc...
1618,A noninflammatory mRNA vaccine for treatment o...,"['Christina Krienke', 'Laura Kolb', 'Elif Dike...","Science (New York, N.Y.)",2021,Non-Genetically Encoded Epitopes Are Relevant ...,['Atkinson M.A. The Pathogenesis and Natural H...


In [12]:
df.reset_index(drop=True,inplace=True)

In [14]:
df.to_csv('final.csv',index=False)